В 6 модуле мы обучали логистическую регрессию для классификации людей в группу риска ишемической болезни сердца в 10-летней перспрективе по датасету framingham.csv.

Если вы помните, модель получилась плохая: несмотря на довольно большую долю верно классифицированных пациентов (около 85%), она очень плохо определяла пациентов группы риска. Чувствительность была нулевая или почти нулевая, а ошибка 2 рода (ложно-отрицательные результаты среди пациентов группы риска) большая.

Для медицинского теста это плохо, так как врач пропустит много пациентов с высоким риском заболеть.

Проблема заключается вот в чем: в обучающей выборке здоровых пациентов намного больше, чем больных. В этом случае классификатору "выгодно" обучиться так, чтобы хорошо определять бОльший класс, так как он тогда чаще будет угадывать.

Попробуем улучшить работу классификатора двумя способами: настроив веса в логистической регрессии и сделав undersampling здоровых пациентов в обучающей выборке.

In [ ]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
%matplotlib inline

In [ ]:
# Импортируем датасет и избавимся от нулевых строк
df = pd.read_csv('framingham.csv')
df.dropna(axis=0,inplace=True) #избавляемся от строчек с пропущенными значениями

# разбиваем датафрейм на две части, dfx - параметры, dfy - целевая переменная. 
dfx = df.drop('TenYearCHD', axis = 1)
dfy = df[['TenYearCHD']] 

# разбиваем датасет на train и test выборку в соотношениии 80% train / 20% test случайным образом
# фиксируем random_state
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.2, random_state=17) 

1 Способ

Его идея заключается в том, что мы добавим на этапе обучения бОльший штраф за ошибки для более редкого класса, тем самым увеличивая чувствительность классификатора к этому классу. 

Среди параметров LogisticRegression есть class_weight. Он может иметь 3 состояния:

1. class_weight=None означает, что мы обучаем регрессию как обычно, без доп. настроек. Так мы делали в практике 6 модуля 
2. class_weight='balanced' задает веса обратно пропорционально количеству элементов в каждом классе. Например, если в выборке будет 1000 здоровых пациентов и 100 больных, то веса будут относиться как 1:10. В описании параметров на https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression есть формула, по которой считаются коэффициенты.
3. class_weight=dict - можно задать веса самостоятельно с формате словаря.

Для начала давайте попробуем, как работает логистическая регрессия с весами

In [ ]:
lm = linear_model.LogisticRegression(solver='liblinear', class_weight='balanced') 
# обучаем
model = lm.fit(X_train, y_train.values.ravel()) 
# сделаем prediction классов на всей тестовой выборке
y_pred = lm.predict(X_test)

In [ ]:
# строим confusion matrix - таблицу правильных и неправильных предсказаний
# можно увидеть, что она ведет себя намного лучше, чем для модели без весов!
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

Как видите, мы сущесвтенно улучшили чувствительность классификатора.

Давайте посмотрим на метрики качества:

In [ ]:
TN = cnf_matrix[0,0] # True Negative
TP = cnf_matrix[1,1] # True Positive
FN = cnf_matrix[1,0] # False Negative
FP = cnf_matrix[0,1] # False Positive
    
Ac = lm.score(X_test, y_test)
Sens = TP/(TP+FN) 
Sp = TN/(TN+FP)
P = TP/(TP+FP)
typeI = FP/(FP+TN)
typeII = FN/(FN+TP)
    
print('Accuracy: ', Ac)
print('Sensitivity: ', Sens)
print('Specificity: ', Sp)
print('Pricision: ', P)
print('Type I error rate: ', typeI)
print('Type II error rate: ', typeII)

Accuracy модели стала меньше, чем в простом случае, но зато ошибка второго рода тоже уменьшилась, что для нас в данном случае важнее.

In [ ]:
# чтобы не делать 100500 копипастов, создадим функцию print_logit_scores
# которая будет обучать регрессию заданным способом и выводить разные метрики качества

def print_logit_scores(data_train, target_train, data_test, target_test, model_type, weights):
    
    # data_train, target_train, data_test, target_test - это обучающие и тестовые данные
    # model_type задает один из 3 типов обучения: 'n' - None, 'b' - balanced, 'w' - заданные пользователем веса
    # w - вектор весов. Используется только для model_type = 'w'
    
    if (model_type == 'n'): # обучаем с равными весами
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight=None)    
    elif (model_type == 'b'): # балансируем веса, как предлагают разработчики sklearn
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight='balanced')
    elif (model_type == 'w'): # балансируем веса самостоятельно
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight={0:weights[0], 1:weights[1]}) 

    # обучаем
    model = lm.fit(data_train, target_train.values.ravel()) 

    # сделаем prediction классов на всей тестовой выборке
    target_pred = lm.predict(data_test)

    # строим confusion matrix - таблицу правильных и неправильных предсказаний
    cnf_matrix = metrics.confusion_matrix(target_test, target_pred)

    TN = cnf_matrix[0,0] # True Negative
    TP = cnf_matrix[1,1] # True Positive
    FN = cnf_matrix[1,0] # False Negative
    FP = cnf_matrix[0,1] # False Positive
    
    Ac = lm.score(data_test, target_test)
    Sens = TP/(TP+FN) 
    Sp = TN/(TN+FP)
    P = TP/(TP+FP)
    typeI = FP/(FP+TN)
    typeII = FN/(FN+TP)
    
    print('Accuracy: ', Ac)
    print('Sensitivity: ', Sens)
    print('Specificity: ', Sp)
    print('Precision: ', P)
    print('Type I error rate: ', typeI)
    print('Type II error rate: ', typeII)
    
    return [Ac,Sens,Sp,P,typeI,typeII] # возвращаем список метрик

интуитивно хочется поделить веса обратно пропорционально количеству элементов в классе, оставив сумму 1

In [ ]:
share = y_train['TenYearCHD'].value_counts()
w0 = share[1]/(share[0]+share[1])
w = np.array([w0,1-w0])
w

разработчики sklearn предлагают балансировать веса по другому правилу. При этом они тоже будут обратно пропорциональны количествую элементов, но сумма будет уже не 1

In [ ]:
np.bincount(y_train['TenYearCHD']) # считает количество вхождений 0 и 1 в y_train['TenYearCHD']
w_b = y_train.shape[0]/ (2*np.bincount(y_train['TenYearCHD']))
w_b

In [ ]:
# Давайте убедимся, что отношения весов действительно одинаковые
# При этом бОльший по размеру класс (нулевой, то есть здоровые пациенты) имеет мЕньший вес
print('отношение интуитивных весов: ', w[0]/w[1])
print('отношение balanced весов: ', w_b[0]/w_b[1])

In [ ]:
# сравним 
print('ручная балансировка по правилу balanced')
m1 = print_logit_scores(X_train, y_train, X_test, y_test, 'w', w_b) # ручная балансировка по правилу balanced
print('\n')
print ('встроенная балансировка по правилу balanced')
m2 = print_logit_scores(X_train, y_train, X_test, y_test, 'b', w) # встроенная балансировка по правилу balanced
print('\n')
print ('без балансировки весов')
m3 = print_logit_scores(X_train, y_train, X_test, y_test, 'n', w) # без балансировки весов

2 Способ

Его идея заключается в том, чтобы уравнять доли "здоровых" и "больных" в обучающей выборке.

Каким образом?

Очень просто: из всех "здоровых" пациентов в обучающей выборке сделам подвыборку того же размера, сколько у нас "больных". Например, если в обучающей выборке 1000 "здоровых" и 100 "больных", то мы из этой 1000 случайным образом выберем 100. Это называется undersampling

In [ ]:
# Нам понадобится новая библиотека imblearn
# в моей версии Anaconda (2019.03) она не предустановлена
# возможно, вам тоже нужно установить ее самостоятельно
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# освежите в памяти, что показывает share и что значит share[1]
# параметр ratio в RandomUnderSampler задается словарем: 
# 1: - желаемое количество объектов класса 1
# 0: - желаемое количество объектов класса 0

# задаем параметры выборки:
sampler = RandomUnderSampler(ratio={1: share[1], 0: share[1]})

# сам unpersampling выполняется здесь:
X_train_under_np, y_train_under_np = sampler.fit_sample(X_train, y_train)

# преобразуем в DataFrame, чтобы скормить логистической регрессии
X_train_under = pd.DataFrame(X_train_under_np)
y_train_under = pd.DataFrame(y_train_under_np)

In [ ]:
# вычисляем качество модели с undersampling
# как вы думаете, почему в качестве model_type здесь можно взять 'n'?
m_u = print_logit_scores(X_train_under, y_train_under, X_test, y_test, 'n', w)

In [ ]:
# сравните результаты со встроенной балансировкой на всей обучающей выборке
# как вы думаете, какой есть существенный недостаток у undersampling по сравнению с балансировкой весов?
m2 = print_logit_scores(X_train, y_train, X_test, y_test, 'b', w)